# 🔍 Validação e Interpretação do Modelo de Dengue

Este notebook explica:
1. **O que o modelo faz** e para que serve
2. **Como confiar nos resultados** (métricas e validação)
3. **Visualizações** para confirmar a qualidade

---

## 📚 O que é este modelo?

É um **modelo de regressão** que prevê **quantos casos de dengue** ocorrerão em um município em uma determinada semana, baseado em:
- Dados climáticos (temperatura, precipitação)
- Histórico de casos (semanas anteriores)
- Características do município (população, região)

### Para que serve?
- **Alertas antecipados**: Prever surtos antes que aconteçam
- **Alocação de recursos**: Saber onde enviar equipes de saúde
- **Planejamento**: Preparar hospitais para demanda

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Bibliotecas carregadas!")

In [ ]:
# Carregar modelo e dados
project_root = Path('..').resolve()

model = joblib.load(project_root / 'models/dengue_model.joblib')
with open(project_root / 'models/model_metadata.json') as f:
    metadata = json.load(f)

print(f"📊 Modelo: {metadata['model_type']}")
print(f"   R² no teste: {metadata['metrics']['test']['r2']:.4f}")
print(f"   MAE no teste: {metadata['metrics']['test']['mae']:.1f} casos")

In [ ]:
# Carregar dados Gold
data_path = project_root / 'data/gold/gold_dengue_clima'
parquet_files = list(data_path.rglob('*.parquet'))

dfs = []
for f in parquet_files:
    temp_df = pd.read_parquet(f)
    if 'uf' not in temp_df.columns:
        parts = [p for p in f.parts if p.startswith('uf=')]
        if parts:
            temp_df['uf'] = parts[0].replace('uf=', '')
    dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)
print(f"📂 Dados carregados: {len(df):,} registros")

---

## 📈 Como interpretar as métricas?

### R² (Coeficiente de Determinação)
- **O que é**: Quanto da variação nos casos o modelo consegue explicar
- **Interpretação**:
  - `R² = 0.95` → Modelo explica 95% da variação ✅
  - `R² = 0.50` → Modelo explica 50% (médio) ⚠️
  - `R² < 0.30` → Modelo fraco ❌

### MAE (Erro Absoluto Médio)
- **O que é**: Em média, quantos casos o modelo erra
- **Interpretação**:
  - `MAE = 11` → Em média, erra por 11 casos
  - Se o município tem 100 casos reais, o modelo prevê entre 89-111

---

## ✅ Validação 1: Cross-Validation Temporal

**Por que isso é importante?**

Em séries temporais, não podemos usar o futuro para prever o passado. O Cross-Validation Temporal simula isso:
- Treina em semanas 1-10, testa na 11
- Treina em semanas 1-20, testa na 21
- etc.

Se o R² se mantém alto em todos os folds, o modelo é confiável.

In [ ]:
# Preparar dados para validação
model_df = df.copy()

# Criar features de lag
model_df = model_df.sort_values(['geocode', 'ano_epidemiologico', 'semana_epidemiologica'])
for lag in [1, 2, 3, 4]:
    model_df[f'casos_lag{lag}'] = model_df.groupby('geocode')['casos_notificados'].shift(lag)
model_df['casos_media_4sem'] = model_df.groupby('geocode')['casos_notificados'].transform(
    lambda x: x.shift(1).rolling(4, min_periods=1).mean()
)

# Encoding
model_df['semana_sin'] = np.sin(2 * np.pi * model_df['semana_epidemiologica'] / 53)
model_df['semana_cos'] = np.cos(2 * np.pi * model_df['semana_epidemiologica'] / 53)
model_df['geocode_hash'] = model_df['geocode'].apply(lambda x: hash(str(x)) % 1000)

region_map = {
    'AC': 0, 'AM': 0, 'AP': 0, 'PA': 0, 'RO': 0, 'RR': 0, 'TO': 0,
    'AL': 1, 'BA': 1, 'CE': 1, 'MA': 1, 'PB': 1, 'PE': 1, 'PI': 1, 'RN': 1, 'SE': 1,
    'DF': 2, 'GO': 2, 'MS': 2, 'MT': 2,
    'ES': 3, 'MG': 3, 'RJ': 3, 'SP': 3,
    'PR': 4, 'RS': 4, 'SC': 4
}
model_df['regiao'] = model_df['uf'].map(region_map).fillna(-1).astype(int)

In [ ]:
# Cross-validation temporal
features = metadata['features']
clean_df = model_df[features + ['casos_notificados']].dropna()

X = clean_df[features]
y = clean_df['casos_notificados']

# 5 splits temporais
tscv = TimeSeriesSplit(n_splits=5)

scores = []
for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    scores.append({'fold': fold, 'r2': r2, 'mae': mae})
    print(f"Fold {fold}: R² = {r2:.4f}, MAE = {mae:.1f}")

scores_df = pd.DataFrame(scores)
print(f"\n📊 Média: R² = {scores_df['r2'].mean():.4f} (±{scores_df['r2'].std():.4f})")

In [ ]:
# Gráfico de Cross-Validation
fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.bar(scores_df['fold'], scores_df['r2'], color='steelblue', edgecolor='white')
ax.axhline(scores_df['r2'].mean(), color='red', linestyle='--', lw=2, label=f"Média: {scores_df['r2'].mean():.3f}")
ax.axhline(0.9, color='green', linestyle=':', lw=2, alpha=0.7, label='Limite bom (0.9)')

ax.set_xlabel('Fold (Divisão Temporal)', fontsize=12)
ax.set_ylabel('R² Score', fontsize=12)
ax.set_title('✅ Cross-Validation Temporal (5 Folds)', fontsize=14)
ax.set_ylim(0, 1)
ax.legend()

# Adicionar valores nas barras
for bar, score in zip(bars, scores_df['r2']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{score:.3f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETAÇÃO:")
if scores_df['r2'].mean() > 0.9:
    print("   ✅ Modelo EXCELENTE! R² consistente acima de 0.9 em todos os folds.")
elif scores_df['r2'].mean() > 0.7:
    print("   ⚠️ Modelo BOM, mas com espaço para melhoria.")
else:
    print("   ❌ Modelo precisa de mais trabalho.")

---

## ✅ Validação 2: Real vs Predito

O gráfico ideal mostra pontos alinhados na diagonal (y = x).

In [ ]:
# Retreinar com todos os dados para visualização
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calcular métricas
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
# Gráfico: Real vs Predito
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
ax1 = axes[0]
ax1.scatter(y_test, y_pred, alpha=0.3, s=10, c='steelblue')
max_val = max(y_test.max(), y_pred.max())
ax1.plot([0, max_val], [0, max_val], 'r-', lw=3, label='Ideal (predito = real)')
ax1.set_xlabel('Casos Reais', fontsize=12)
ax1.set_ylabel('Casos Preditos', fontsize=12)
ax1.set_title(f'Real vs Predito (R² = {r2:.3f})', fontsize=14)
ax1.legend(fontsize=11)

# Zoom em casos menores
ax2 = axes[1]
mask = (y_test < 500) & (y_pred.flatten() < 500)
ax2.scatter(y_test[mask], y_pred[mask], alpha=0.3, s=15, c='coral')
ax2.plot([0, 500], [0, 500], 'r-', lw=3, label='Ideal')
ax2.set_xlabel('Casos Reais', fontsize=12)
ax2.set_ylabel('Casos Preditos', fontsize=12)
ax2.set_title('Zoom: Casos < 500 (maioria)', fontsize=14)
ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETAÇÃO:")
print("   - Quanto mais alinhados à linha vermelha, melhor o modelo")
print(f"   - R² = {r2:.3f} → O modelo explica {r2*100:.0f}% da variação nos casos")

---

## ✅ Validação 3: Distribuição dos Erros

Os erros devem estar centrados em zero (sem viés).

In [ ]:
# Calcular erros
errors = y_test.values - y_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
ax1 = axes[0]
ax1.hist(errors, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
ax1.axvline(0, color='red', linestyle='--', lw=2, label='Zero (sem erro)')
ax1.axvline(errors.mean(), color='orange', linestyle='-', lw=2, label=f'Média: {errors.mean():.1f}')
ax1.set_xlabel('Erro (Real - Predito)', fontsize=12)
ax1.set_ylabel('Frequência', fontsize=12)
ax1.set_title('Distribuição dos Erros', fontsize=14)
ax1.legend()

# Boxplot por região
ax2 = axes[1]
error_df = pd.DataFrame({'erro': errors, 'regiao': X_test['regiao'].values})
region_names = {0: 'Norte', 1: 'Nordeste', 2: 'C-Oeste', 3: 'Sudeste', 4: 'Sul'}
error_df['regiao_nome'] = error_df['regiao'].map(region_names)
sns.boxplot(data=error_df, x='regiao_nome', y='erro', ax=ax2, palette='Set2')
ax2.axhline(0, color='red', linestyle='--', lw=2)
ax2.set_xlabel('Região', fontsize=12)
ax2.set_ylabel('Erro', fontsize=12)
ax2.set_title('Erro por Região', fontsize=14)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETAÇÃO:")
print(f"   - Média do erro: {errors.mean():.2f} (quanto mais perto de 0, melhor)")
print(f"   - Desvio padrão: {errors.std():.2f}")
if abs(errors.mean()) < 5:
    print("   ✅ Modelo sem viés significativo!")

---

## ✅ Validação 4: Feature Importance

Quais variáveis o modelo mais usa para prever?

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importância': model.feature_importances_
}).sort_values('Importância', ascending=True)

# Categorizar features
def categorize(f):
    if 'casos' in f:
        return 'Casos (lag)'
    elif 'inmet' in f:
        return 'Clima'
    elif 'semana' in f or 'ano' in f:
        return 'Temporal'
    else:
        return 'Geográfico'

importance_df['Categoria'] = importance_df['Feature'].apply(categorize)

# Cores por categoria
color_map = {'Casos (lag)': '#2ecc71', 'Clima': '#3498db', 'Temporal': '#f1c40f', 'Geográfico': '#e74c3c'}
colors = [color_map[cat] for cat in importance_df['Categoria']]

plt.figure(figsize=(12, 8))
bars = plt.barh(importance_df['Feature'], importance_df['Importância'], color=colors)
plt.xlabel('Importância', fontsize=12)
plt.title('🎯 Importância das Features', fontsize=14)

# Legenda
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color_map[k], label=k) for k in color_map]
plt.legend(handles=legend_elements, loc='lower right', fontsize=11)

plt.tight_layout()
plt.show()

# Resumo por categoria
print("\n📊 Importância por Categoria:")
cat_importance = importance_df.groupby('Categoria')['Importância'].sum().sort_values(ascending=False)
for cat, imp in cat_importance.items():
    print(f"   {cat}: {imp*100:.1f}%")

---

## ✅ Validação 5: Análise Temporal

O modelo funciona bem ao longo do tempo?

In [ ]:
# Adicionar predições ao dataframe de teste
test_df = clean_df.iloc[X_test.index].copy()
test_df['predito'] = y_pred
test_df['erro'] = test_df['casos_notificados'] - test_df['predito']

# Agregar por semana
weekly = test_df.groupby('semana_epidemiologica').agg({
    'casos_notificados': 'sum',
    'predito': 'sum',
    'erro': 'mean'
}).reset_index()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Casos reais vs preditos por semana
ax1 = axes[0]
ax1.plot(weekly['semana_epidemiologica'], weekly['casos_notificados'], 
         'b-', lw=2, marker='o', label='Real')
ax1.plot(weekly['semana_epidemiologica'], weekly['predito'], 
         'r--', lw=2, marker='s', label='Predito')
ax1.set_xlabel('Semana Epidemiológica', fontsize=12)
ax1.set_ylabel('Total de Casos', fontsize=12)
ax1.set_title('Casos Reais vs Preditos por Semana', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Erro por semana
ax2 = axes[1]
colors = ['green' if e < 0 else 'red' for e in weekly['erro']]
ax2.bar(weekly['semana_epidemiologica'], weekly['erro'], color=colors, alpha=0.7)
ax2.axhline(0, color='black', linestyle='-', lw=1)
ax2.set_xlabel('Semana Epidemiológica', fontsize=12)
ax2.set_ylabel('Erro Médio', fontsize=12)
ax2.set_title('Erro Médio por Semana (verde = subestimou, vermelho = superestimou)', fontsize=14)

plt.tight_layout()
plt.show()

---

## ✅ Validação 6: Exemplos Concretos

Vamos ver casos específicos para entender melhor.

In [ ]:
# Adicionar nome do município se disponível
if 'nome_municipio' in model_df.columns:
    test_df = test_df.merge(
        model_df[['geocode', 'nome_municipio', 'uf']].drop_duplicates(),
        left_on='geocode_hash', right_on=model_df['geocode'].apply(lambda x: hash(str(x)) % 1000),
        how='left'
    )

# Top 10 melhores predições (menor erro percentual)
test_df['erro_pct'] = np.abs(test_df['erro']) / (test_df['casos_notificados'] + 1) * 100

print("✅ TOP 10 - Melhores Predições (menor erro):")
best = test_df.nsmallest(10, 'erro_pct')[['casos_notificados', 'predito', 'erro']]
best.columns = ['Casos Reais', 'Predição', 'Erro']
print(best.to_string(index=False))

print("\n❌ TOP 10 - Piores Predições (maior erro):")
worst = test_df[test_df['casos_notificados'] > 50].nlargest(10, 'erro_pct')[['casos_notificados', 'predito', 'erro']]
worst.columns = ['Casos Reais', 'Predição', 'Erro']
print(worst.to_string(index=False))

---

## 📋 Resumo da Validação

Execute a célula abaixo para ver um resumo completo.

In [ ]:
print("=" * 60)
print("📋 RESUMO DA VALIDAÇÃO")
print("=" * 60)

print(f"\n🎯 Performance Geral:")
print(f"   R² = {r2:.4f} ({r2*100:.1f}% da variação explicada)")
print(f"   MAE = {mae:.1f} casos de erro médio")

print(f"\n✅ Cross-Validation:")
print(f"   Média R² = {scores_df['r2'].mean():.4f} (±{scores_df['r2'].std():.4f})")
print(f"   Modelo estável ao longo do tempo: {'SIM ✅' if scores_df['r2'].std() < 0.1 else 'NÃO ⚠️'}")

print(f"\n📊 Viés:")
print(f"   Erro médio = {errors.mean():.2f}")
print(f"   Modelo sem viés: {'SIM ✅' if abs(errors.mean()) < 5 else 'NÃO ⚠️'}")

print(f"\n🔑 Features mais importantes:")
for _, row in importance_df.tail(3).iloc[::-1].iterrows():
    print(f"   • {row['Feature']}: {row['Importância']*100:.1f}%")

print("\n" + "=" * 60)
if r2 > 0.9 and scores_df['r2'].std() < 0.1:
    print("🏆 CONCLUSÃO: Modelo CONFIÁVEL para uso em produção!")
elif r2 > 0.7:
    print("⚠️ CONCLUSÃO: Modelo BOM, mas pode ser melhorado.")
else:
    print("❌ CONCLUSÃO: Modelo precisa de mais trabalho.")
print("=" * 60)